# Spike Train Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/nancy/projects/reward_competition_extention'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

## Inputs & Data

Explanation of each input and where it comes from.

In [7]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.read_pickle(os.path.join(git_root, "notebooks/export/proc/rce_pilot_2_lfp_traces_sleap_spikes.pkl"))

OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs



In [8]:
TRIAL_LABELS_DF = pd.read_excel("./rce_per_trial_labeling.xlsx")
TRIAL_LABELS_DF = TRIAL_LABELS_DF.rename(columns={col: col.strip().lower() for col in TRIAL_LABELS_DF.columns})
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition"])
TRIAL_LABELS_DF["time_stamp_index"] = TRIAL_LABELS_DF["time_stamp_index"].astype(np.int64)


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [9]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [10]:
FULL_LFP_TRACES_PKL = "{}_lfp_traces_sleap_spikes.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [11]:
TRIAL_LABELS_DF

,unnamed: 0,recording_file,video_file,time_stamp_index,video_frame,reward_frame,condition,competition_closeness,notes,time,subject_info,din,state,session_dir,video_number
84,2402.0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,1734.0,1814.0,rewarded,NaN,NaN,6310663.0,6_1_top_2_base_3,dio_ECU_Din1,1.0,20221202_134600_omission_and_competition_subje...,1.0
85,2403.0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,3728.0,3808.0,rewarded,NaN,NaN,7910662.0,6_1_top_2_base_3,dio_ECU_Din1,1.0,20221202_134600_omission_and_competition_subje...,1.0
86,2404.0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,5972.0,6052.0,rewarded,NaN,NaN,9710660.0,6_1_top_2_base_3,dio_ECU_Din1,1.0,20221202_134600_omission_and_competition_subje...,1.0
87,2405.0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,7966.0,8046.0,omission,NaN,NaN,11310658.0,6_1_top_2_base_3,dio_ECU_Din1,1.0,20221202_134600_omission_and_competition_subje...,1.0
88,2406.0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,9836.0,9916.0,rewarded,NaN,NaN,12810657.0,6_1_top_2_base_3,dio_ECU_Din1,1.0,20221202_134600_omission_and_competition_subje...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032,174.0,20230628_111202_standard_comp_to_novel_agent_D...,20230628_111202_standard_comp_to_novel_agent_D...,24708627,24552.0,24632.0,1.2,Subj 2 then Subj 1,NaN,28120638.0,1-2vs1-1and2-1,dio_ECU_Din1,1.0,20230628_111202_standard_comp_to_novel_agent_D...,1.0
2033,175.0,20230628_111202_standard_comp_to_novel_agent_D...,20230628_111202_standard_comp_to_novel_agent_D...,26308645,26149.0,26229.0,1.1,Subj 1 then Subj 2,NaN,29720656.0,1-2vs1-1and2-1,dio_ECU_Din1,1.0,20230628_111202_standard_comp_to_novel_agent_D...,1.0
2034,176.0,20230628_111202_standard_comp_to_novel_agent_D...,20230628_111202_standard_comp_to_novel_agent_D...,27708663,27547.0,27627.0,1.1,Close Call,NaN,31120674.0,1-2vs1-1and2-1,dio_ECU_Din1,1.0,20230628_111202_standard_comp_to_novel_agent_D...,1.0
2035,177.0,20230628_111202_standard_comp_to_novel_agent_D...,20230628_111202_standard_comp_to_novel_agent_D...,29908690,29743.0,29823.0,1.1,Close Call,NaN,33320701.0,1-2vs1-1and2-1,dio_ECU_Din1,1.0,20230628_111202_standard_comp_to_novel_agent_D...,1.0


In [12]:
# grouped = TRIAL_LABELS_DF.groupby(['recording_file', 'video_file'])['time_stamp_index'].apply(lambda x: sorted(list(x))).reset_index()


grouped = TRIAL_LABELS_DF.groupby(['session_dir', 'video_file']).agg({'time_stamp_index': lambda x: list(x), 'condition': lambda x: list(x), 'competition_closeness': lambda x: list(x)}).reset_index()


In [13]:
grouped["video_file"] = grouped["video_file"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [14]:
VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.merge(VIDEO_TO_FRAME_AND_SUBJECT_DF, grouped, how='inner', left_on=['session_dir', 'video_name'], right_on=['session_dir', 'video_file'])

In [18]:
VIDEO_TO_FRAME_AND_SUBJECT_DF = VIDEO_TO_FRAME_AND_SUBJECT_DF.rename(columns={'time_stamp_index': 'trial_time_stamps', 'condition': 'trial_label', 'competition_closeness': 'trial_competition_closeness'}, errors="ignore")

In [20]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_time_stamps"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_time_stamps"].apply(lambda x: np.array(x))
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"].apply(lambda x: np.array(x))
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].apply(lambda x: np.array(x))

- Converting the trial label to win or lose based on who won the trial

In [24]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.columns

Index(['agent', 'cohort', 'session_dir', 'sleap_filename', 'sleap_fileroot',
       'start_stop_frame', 'tone_frames', 'port_entry_frames', 'box_top_left',
       'box_bottom_left', 'filtered_subject_locations',
       'filtered_agent_locations', 'video_name', 'notes', 'session_path',
       'file_path', 'reward_port', 'recording', 'box_bottom_right',
       'box_top_right', 'subject', 'tracked_subject', 'all_subjects',
       'in_video_subjects', 'first_timestamp', 'raw_timestamps',
       'tone_timestamps', 'port_entry_timestamps', 'start_stop_timestamps',
       'filtered_lfp_timestamps', 'filtered_video_timestamps',
       'filtered_mpfc_trace', 'filtered_vhpc_trace', 'filtered_bla_trace',
       'filtered_lh_trace', 'filtered_md_trace', 'recording_name',
       'spike_clusters', 'spike_times', 'neuron_average_fr',
       'neuron_average_timestamps', 'video_file', 'trial_time_stamps',
       'trial_label', 'trial_competition_closeness'],
      dtype='object')

In [25]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: 
    
    
    
    "win" if str(x["trial_label"]).strip() == str(x["subject"]) 
             else ("lose" if str(x["trial_label"]) in x["all_subjects"] 
                   else x["trial_label"]), axis=1)

In [31]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"].iloc[0]

array([1.2, 1.1, 1.1, 1.2, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.2, 1.1, 1.1, 1.2, 1.1,
       1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1,
       1.1, 1.1, 1.1])

- Adding the competition closeness as a column

In [ ]:
competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else "comp" if type(k) is str else np.nan for k in VIDEO_TO_FRAME_AND_SUBJECT_DF["competition_closeness"].unique()}

In [ ]:
competition_closeness_map

{nan: nan,
 'Subj 1 Only': 'non_comp',
 'Subj 2 blocking Subj 1': 'comp',
 'Subj 1 then Subj 2': 'comp',
 'Subj 1 blocking Subj 2': 'comp',
 'Subj 2 Only': 'non_comp',
 'Subj 2 then Subj 1': 'comp',
 'Close Call': 'comp'}

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["competition_closeness"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["competition_closeness"].map(competition_closeness_map)

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["competition_closeness"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: "_".join([str(x["trial_outcome"]), str(x["competition_closeness"])]).strip("nan").strip("_"), axis=1)

In [21]:
VIDEO_TO_FRAME_AND_SUBJECT_DF

,agent,cohort,session_dir,sleap_filename,sleap_fileroot,start_stop_frame,tone_frames,port_entry_frames,box_top_left,box_bottom_left,...,filtered_md_trace,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps,video_file,trial_time_stamps,trial_label,trial_competition_closeness
0,1.2,2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"(1, 39500)","[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[29.623370221193532, 10.85380707317621]","[30.23007505102494, 34.598706975722735]",...,"[-2.3028033, -2.0405018, -1.7475417, -1.429032...",20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223, 2367433, 2569212, 3969647, 3973243, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230617_115521_standard_comp_to_omission_D1_s...,"[1030879, 3430878, 5730880, 7530877, 8630880, ...","[1.2, 1.1, 1.1, 1.2, 1.1, 1.1, 1.1, 1.1, 1.1, ...","[Subj 2 blocking Subj 1, Subj 1 blocking Subj ..."
1,1.1,2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"(1, 39500)","[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[29.623370221193532, 10.85380707317621]","[30.23007505102494, 34.598706975722735]",...,"[-0.62384546, -0.897785, -1.1164762, -0.982959...",20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958, 5233, 6331, 7475, 8599, 10550, 23459, 2...","[[1.0, 0.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230617_115521_standard_comp_to_omission_D1_s...,"[1030879, 3430878, 5730880, 7530877, 8630880, ...","[1.2, 1.1, 1.1, 1.2, 1.1, 1.1, 1.1, 1.1, 1.1, ...","[Subj 2 blocking Subj 1, Subj 1 blocking Subj ..."
2,1.4,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"(1, 32316)","[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[28.94954695212123, 11.559275766924861]","[29.794023445649177, 35.275466614423074]",...,"[0.3162679, 0.49376523, 0.4066302, 0.13715701,...",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120...","[[2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 3.0, 0.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[1.4, 1.1, 1.1, 1.4, 1.1, 1.4, 1.4, 1.4, 1.1, ...","[Subj 2 Only, Subj 1 Only, Subj 1 Only, Subj 2..."
3,1.1,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"(1, 32316)","[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[28.94954695212123, 11.559275766924861]","[29.794023445649177, 35.275466614423074]",...,"[0.7159058, -0.05324919, -0.6389903, 0.0, 0.44...",20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[1.4, 1.1, 1.1, 1.4, 1.1, 1.4, 1.4, 1.4, 1.1, ...","[Subj 2 Only, Subj 1 Only, Subj 1 Only, Subj 2..."
4,1.2,2,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,"(2027, 32239)"

In [16]:
raise ValueError("This is not done yet")

ValueError: This is not done yet

In [ ]:
ALL_PHY_DIR

In [ ]:
recording_to_cluster_info = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        recording_to_cluster_info[recording_basename] = pd.read_csv(file_path, sep="\t")
    except Exception as e:
        print(e)

In [ ]:
recording_to_cluster_info[list(recording_to_cluster_info.keys())[1]].head()

- Combining all the unit info dataframes and adding the recording name

In [ ]:
recording_to_cluster_info_df = pd.concat(recording_to_cluster_info, names=['recording_name']).reset_index(level=1, drop=True).reset_index()


In [ ]:
recording_to_cluster_info_df.head()

- Filtering for the good units

In [ ]:
good_unit_cluster_info_df = recording_to_cluster_info_df[recording_to_cluster_info_df["group"] == "good"].reset_index(drop=True)

In [ ]:
good_unit_cluster_info_df.head()

In [ ]:
recording_to_good_unit_ids = good_unit_cluster_info_df.groupby('recording_name')['cluster_id'].apply(list).to_dict()

- A list of all the unit IDs that each spike came from in order
    - First item is first spike, second item is second spike, etc.

In [ ]:
recording_to_spike_clusters = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        recording_to_spike_clusters[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [ ]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]]

In [ ]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]].shape

- The times that all the spikes happened

In [ ]:
recording_to_spike_times = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        recording_to_spike_times[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [ ]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]]

In [ ]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]].shape

### Combining everything into a dataframe

In [ ]:
recording_to_spike_df = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        cluster_info_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        cluster_info_df = pd.read_csv(cluster_info_path, sep="\t")

        spike_clusters_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        spike_clusters = np.load(spike_clusters_path)
        
        spike_times_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        spike_times = np.load(spike_times_path)

        spike_df = pd.DataFrame({'spike_clusters': spike_clusters, 'spike_times': spike_times.T[0]})

        merged_df = spike_df.merge(cluster_info_df, left_on='spike_clusters', right_on='cluster_id', how="left")
        merged_df["recording_name"] = recording_basename

        merged_df["timestamp_isi"] = merged_df.groupby('spike_clusters')["spike_times"].diff()
        merged_df["current_isi"] = merged_df["timestamp_isi"] / SAMPLING_RATE
        
        if not merged_df.empty:
            recording_to_spike_df[recording_basename] = merged_df
       
    except Exception as e:
        print(e)

In [ ]:
cluster_info_df.head()

In [ ]:
spike_times

In [ ]:
spike_clusters

- Combining the spike time df for all recordings

In [ ]:
all_spike_time_df = pd.concat(recording_to_spike_df.values())

In [ ]:
all_spike_time_df = all_spike_time_df[all_spike_time_df["group"] == "good"].reset_index(drop=True)

In [ ]:
all_spike_time_df.head()

In [ ]:
all_spike_time_df.tail()

## Grouping all the neurons by recording

In [ ]:
# Grouping all spike times by neuron and recording

grouped_df = all_spike_time_df.groupby(['spike_clusters', 'recording_name'])["spike_times"].apply(lambda x: sorted(list(x))).reset_index()
grouped_df = grouped_df.sort_values(by=['recording_name', 'spike_clusters']).reset_index(drop=True)
grouped_df = grouped_df.groupby('recording_name').agg({'spike_clusters': lambda x: list(x), 'spike_times': lambda x: list(x)}).reset_index()

In [ ]:
grouped_df

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.merge(VIDEO_TO_FRAME_AND_SUBJECT_DF, grouped_df, left_on='recording', right_on="recording_name", how='inner')

## Calculating the firing rates

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.columns

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.head()

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: np.array([neuron.spikes.calculate_rolling_avg_firing_rate(np.array(times), stop_time=x["start_stop_timestamps"][1], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[0] for times in x["spike_times"]]), axis=1)

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_timestamps"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: neuron.spikes.calculate_rolling_avg_firing_rate(x["spike_times"][0], stop_time=x["start_stop_timestamps"][1], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[1], axis=1)

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: x["neuron_average_fr"] * SPIKE_WINDOW, axis=1)

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5,:10]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5,10:20]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5,20:30]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5, 30:]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_timestamps"].iloc[0].shape

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"].iloc[0].shape

In [ ]:
np.max(VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"].iloc[0])

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError("")